In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv


# Сиды

In [2]:
import os
import random
import numpy as np
import torch
import numpy as np # linear algebra
import pandas as pd 
try:
    # Hugging Face convenience fn; sets Python/Rand, NumPy, Torch seeds
    from transformers import set_seed  
except ImportError:
    set_seed = None

def seed_everything(seed):
    # 1. Python built‑ins
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    # 2. NumPy
    np.random.seed(seed)

    # 3. PyTorch (CPU & GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 4. CuDNN: make deterministic, but may slow you down
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 5. Transformers (if installed)
    if set_seed is not None:
        set_seed(seed)

# call it!
seed_everything(257)

2025-08-06 12:18:00.803559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754482680.981022      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754482681.033774      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Импорты

In [3]:
import torch 
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score

# Загрузка данных

In [4]:
train=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
sample=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')


# А-ля EDA

In [5]:
train

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


# Датасет

In [6]:
zeros = train[train['target'] == 0]
ones  = train[train['target'] == 1]

In [7]:
zeros_sampled = zeros.sample(n=len(ones), random_state=42)
gol = pd.concat([zeros_sampled, ones]).reset_index(drop=True)

In [8]:
gol

,qid,question_text,target
0,c86ab618f85e9b7fc374,Is sadism a coping mechanism for people who ar...,0
1,7d9ea6d66b8866e69240,Is it possible for me as a soul to go outside ...,0
2,f01982d0cd06aba308ed,Do Pet Animal Rescue workers minimize shows of...,0
3,dd8a6b5452a407cea2ac,How do you identify a sonnet and what can we i...,0
4,ef30e73bf0a81a06ccf6,Is there a special place in hell for the likes...,0
...,...,...,...
161615,fffeba722d9b371bd1b9,How is it to have intimate relation with your ...,1
161616,fffee269360dd0d3947a,Why is it when singers have lyrics about voice...,1
161617,ffff0e4ea1bb6e16feec,Do pakis smell of curry and shit?,1
161618,ffff3f0a2449ffe4b9ff,Isn't Trump right after all? Why should the US...,1


In [9]:
small_train = gol.sample(n=100000).reset_index(drop=True)

In [10]:
train_text=small_train['question_text']
test_text=test['question_text']
target=small_train['target']

In [11]:
train_texts, val_texts, train_targets, val_targets=train_test_split(train_text, target, test_size=0.1, stratify=target)

In [12]:
class QuoraDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        # Force lists to avoid pandas indexing quirks
        self.texts  = list(texts)
        self.labels = None if labels is None else list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


# Модель

In [13]:
name='roberta-base'
model=AutoModelForSequenceClassification.from_pretrained(name)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Токенайзер

In [14]:
tokenizer=AutoTokenizer.from_pretrained(name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Делание датасетов

In [15]:
train_dataset=QuoraDataset(train_texts, train_targets, tokenizer, max_len=384)
val_dataset=QuoraDataset(val_texts, val_targets, tokenizer, max_len=384)
test_dataset=QuoraDataset(test_text, [0]*len(test_text), tokenizer, max_len=384)

# Метрика

In [16]:
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=1)
    return {'f1': f1_score(eval_pred.label_ids, preds)}

# Training args

In [17]:
args=TrainingArguments(
    output_dir='gol',
    num_train_epochs=2,
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy='steps',
    eval_steps=500,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    save_strategy='steps',
    greater_is_better=True,
    report_to='none',
    fp16=True

)

# Trainer

In [18]:
trainer=Trainer(
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    
)

# Обучение

In [19]:
trainer.train()

Step,Training Loss,Validation Loss,F1
500,0.389400,0.252815,0.903219
1000,0.271900,0.242053,0.911691
1500,0.252500,0.259091,0.903754
2000,0.242600,0.213824,0.921487
2500,0.234600,0.225751,0.923257
3000,0.208900,0.229174,0.925699
3500,0.174100,0.222628,0.925093
4000,0.176900,0.212811,0.929148
4500,0.168300,0.207032,0.929783
5000,0.167700,0.204548,0.928693


TrainOutput(global_step=5626, training_loss=0.22188754714005704, metrics={'train_runtime': 7632.5574, 'train_samples_per_second': 23.583, 'train_steps_per_second': 0.737, 'total_flos': 3.55199924736e+16, 'train_loss': 0.22188754714005704, 'epoch': 2.0})

# Сабмит

In [20]:
SAVE_DIR = "./quora_ckpt"   # will hold config.json, pytorch_model.bin, vocab, etc.
trainer.save_model(SAVE_DIR)              # saves model weights + config
tokenizer.save_pretrained(SAVE_DIR)       # saves tokenizer files

# ---------- 4. (optional) zip it for upload ----------
import shutil, os, zipfile
ZIP_NAME = "quora_ckpt.zip"
shutil.make_archive(base_name=SAVE_DIR, format='zip', root_dir=SAVE_DIR)

print(f"Checkpoint written to {SAVE_DIR}  and zipped as {ZIP_NAME}")

Checkpoint written to ./quora_ckpt  and zipped as quora_ckpt.zip


In [21]:
#output = trainer.predict(test_dataset)
#logits = output.predictions
#preds = np.argmax(logits, axis=1).astype(int)

#sample['prediction'] = preds
#sample.to_csv('submission.csv', index=False)
